In [3]:
import numpy as np
import torch

1. Tensors (torch.Tensor, Tensor Attributes)

In [39]:
#from python list or sequence
a = [[1_000, 0], [3, 4]]
#dtype int8, unsigned int8, int16, int32, int64, float16, float32, float64, boolean
x = torch.tensor(a, dtype=torch.bool, device='cpu')
x.dtype, x.device, x.stride() #reference to tensor.Storage

(torch.bool, device(type='cpu'), (2, 1))

In [45]:
a = np.array([[1_000, 0], [3, 4]])
q = torch.as_tensor(a)
q[0, 0] = -1
a[0, 0], q.device

(-1, device(type='cpu'))

In [52]:
v = torch.Tensor()
v.dtype #default float32
# 9 cpu constructors with specific dtype
torch.FloatTensor
#x, x.dtype
# 9 gpu constructors with specific dtype
torch.cuda.FloatTensor

(tensor([[1., 2., 3.]], device='cuda:0'),
 torch.float32,
 device(type='cuda', index=0))

In [103]:
a = np.array([[[1, 2, 3], 
               [3, 4, 0]],
                        [[1, 2, 3], 
                        [3, 4, 0]]])
a.shape, a.sum(axis=0), a.sum(axis=1), a.sum(axis=2)

((2, 2, 3), array([[2, 4, 6],
        [6, 8, 0]]), array([[4, 6, 3],
        [4, 6, 3]]), array([[6, 7],
        [6, 7]]))

In [101]:
b = torch.tensor(a)
b.sum(dim=0), b.sum(dim=1)

(tensor([4, 6, 3]), tensor([6, 7]))

2. torch.Autograd

In [81]:
#torch autograd
t = torch.tensor([[1, 2, 3]], dtype=torch.float32, requires_grad=True)
out = t.pow(2).sum()
out.backward()
t.grad

tensor([[2., 4., 6.]])